[Vidéo 13](https://www.youtube.com/watch?v=PQCE9ChuIDY&list=PLeo1K3hjS3uu7CxAacxVndI4bE_o3BDtO&index=13)

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('../data/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size=0.2, random_state=25)

In [ ]:
# Scaling data : Age / 100
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

# Utile pour avoir l'age et l'affordibility à la même échelle (0 < age, affor < 1)
# C'est plus efficace lors de l'apprentissage du réseau de neurone

In [ ]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [ ]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

## Création de notre réseau de neuron


In [ ]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.b = 0

    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.b = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_thresold)

    def predict(self, X_test):
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.b
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age, affordability, y_true, epochs, loss_thresold):
        # w1, w2, bias
        w1 = w2 = 1
        bias = 0
        rate = 0.5 # Learning rate
        n = len(age)

        for i in range(1, epochs + 1):
            weighted_sum = w1 * age + w2 * affordability + bias # 1er partie du neurone
            y_predicted = sigmoid_numpy(weighted_sum) # 2e partie du neurone : Activation

            loss = log_loss(y_true, y_predicted) # Calcul du loss pour les metrics de fin

            # Calcul des nouveaux poids/biais avec la formule de derivation partiel
            w1d = (1 / n) * np.dot(np.transpose(age), (y_predicted-y_true))
            w2d = (1 / n) * np.dot(np.transpose(affordability), (y_predicted - y_true))
            bias_d = np.mean(y_predicted - y_true)

            # Nouveaux poids/biais
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d

            if i%50 == 0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias # pour pouvoir les utiliser dans le prochain epoch


In [ ]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=500, loss_thresold=0.4631)

Epoch:50, w1:1.4890156230318012, w2:1.1025844223976542, bias:-1.2216210542672539, loss:0.5682796009861607
Epoch:100, w1:2.1871944491052364, w2:1.2918774898345378, bias:-1.6533819823304428, loss:0.5395530569489538
Epoch:150, w1:2.8371346977812824, w2:1.3687127378995887, bias:-1.9802293096782038, loss:0.5180278589849453
Epoch:200, w1:3.4316889691263146, w2:1.4037274192103975, bias:-2.252091504673362, loss:0.5008101251292182
Epoch:250, w1:3.972165348838788, w2:1.4235853514989025, bias:-2.489891856859742, loss:0.4867936645415293
Epoch:300, w1:4.462839550361659, w2:1.4385080284751603, bias:-2.703328362713941, loss:0.4752896446882009
Epoch:350, w1:4.9087498356574875, w2:1.4522911082498116, bias:-2.8974674516679864, loss:0.4657872590867513
Epoch:367, w1:5.051047623653049, w2:1.4569794548473887, bias:-2.9596534546250037, loss:0.46293944095888917


In [ ]:
# coef, intercept

# (array([[5.060863 ],
#         [1.4086521]], dtype=float32),
#  array([-2.913703], dtype=float32))

In [ ]:
customModel.predict(X_test_scaled)

,0
2,0.705020
10,0.355836
21,0.161599
11,0.477919
14,0.725586
9,0.828987


In [ ]:
# Imaginer les sortie en fonction des entrée pour voir si elles correspondent
# model.predict(X_test_scaled)

# 1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
# array([[0.7054848 ],
#        [0.3556957 ],
#        [0.16827832],
#        [0.47801185],
#        [0.7260696 ],
#        [0.8294983 ]], dtype=float32)